In [19]:
import time

import pandas as pd
import numpy as np
import torch
import pyro

from numpy.linalg import inv

In [20]:
ratings = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')

In [21]:
unique_userId = ratings.userId.unique()
unique_movieId = ratings.movieId.unique()

ratings['new_user_index'], ratings['new_movie_index'] = 0, 0

for old_id, new_id in zip(unique_userId, range(len(unique_userId))):
    ratings['new_user_index'].iloc[ratings[ratings['userId']==old_id].index.tolist()] = new_id

for old_id, new_id in zip(unique_movieId, range(len(unique_movieId))):
    ratings['new_movie_index'].iloc[ratings[ratings['movieId']==old_id].index.tolist()] = new_id

/Users/nishimura_yuki/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
class PMF:
    def __init__(self, k, prior_mean=0, prior_std=1, epochs=10):
        self.k = k
        self.prior_mean = prior_mean
        self.prior_std = prior_std
        self.epochs = epochs
        
    def fit(self, data):
        ratings = data
        self.I = ratings.userId.nunique()
        self.J = ratings.movieId.nunique()
        self.u = pyro.sample("user_factor", pyro.distributions.Normal(self.prior_mean, self.prior_std), sample_shape=torch.Size([self.I,self.k]))
        self.v = pyro.sample("movie_factor", pyro.distributions.Normal(self.prior_mean, self.prior_std), sample_shape=torch.Size([self.J,self.k]))
        self.data_var = ratings['rating'].var(axis=0)
        
        user_to_movie_dict = {}
        movie_to_user_dict = {}

        for i in range(self.I):
            user_to_movie_dict[i] = ratings[ratings['new_user_index']==i]['new_movie_index'].tolist()
        for j in range(self.J):
            movie_to_user_dict[j] = ratings[ratings['new_movie_index']==j]['new_user_index'].tolist()
            
        for epoch_num in range(self.epochs):
            # Update of all u_i: Takes 110.70313000679016 seconds
            for i in range(len(self.u)):
                v_sum = 0
                mv_sum = 0
                for j in user_to_movie_dict[i]:
                    v_sum += np.outer(self.v[j],self.v[j])
                    mv_sum += float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) * self.v[j]
                self.u[i] = torch.from_numpy(np.dot(inv(self.prior_std * self.data_var * np.identity(self.k) + v_sum), mv_sum))

            # Update of all v_j: Takes 106.21870613098145 seconds
            for j in range(len(self.v)):
                u_sum = 0
                mu_sum = 0
                for i in movie_to_user_dict[j]:
                    u_sum += np.outer(self.u[i],self.u[i])
                    mu_sum += float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) * self.u[i]
                self.v[j] = torch.from_numpy(np.dot(inv(self.prior_std * self.data_var * np.identity(self.k) + u_sum), mu_sum))

            M_prime = torch.mm(self.u,self.v.T)
            error = 0
            for i in range(len(self.u)):
                for j in user_to_movie_dict[i]:
                    error += (float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) - M_prime[i][j])**2
            print('MSE of Epoch ' + str(epoch_num) + ': ' + str(float(error/len(ratings))))

In [35]:
model = PMF(3)

In [ ]:
model.fit(ratings)

MSE of Epoch 0: 11.13010025024414
MSE of Epoch 1: 0.988913357257843


In [33]:
# Defining Parameters/Hyperparameters
I = ratings.userId.nunique()
J = ratings.movieId.nunique()
data_var = ratings['rating'].var(axis=0)
k = 3
mean = 0
std = 1

In [101]:
u = pyro.sample("user_factor", pyro.distributions.Normal(mean, std), sample_shape=torch.Size([I,k]))
v = pyro.sample("movie_factor", pyro.distributions.Normal(mean, std), sample_shape=torch.Size([J,k]))

In [118]:
user_to_movie_dict = {}
movie_to_user_dict = {}

for i in range(I):
    user_to_movie_dict[i] = ratings[ratings['new_user_index']==i]['new_movie_index'].tolist()
for j in range(J):
    movie_to_user_dict[j] = ratings[ratings['new_movie_index']==j]['new_user_index'].tolist()

In [177]:
# Update of all u_i: Takes 110.70313000679016 seconds
for i in range(len(u)):
    v_sum = 0
    mv_sum = 0
    for j in user_to_movie_dict[i]:
        v_sum += np.outer(v[j],v[j])
        mv_sum += float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) * v[j]
    u[i] = torch.from_numpy(np.dot(inv(std * data_var * np.identity(k) + v_sum), mv_sum))

# Update of all v_j: Takes 106.21870613098145 seconds
for j in range(len(v)):
    u_sum = 0
    mu_sum = 0
    for i in movie_to_user_dict[j]:
        u_sum += np.outer(u[i],u[i])
        mu_sum += float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) * u[i]
    v[j] = torch.from_numpy(np.dot(inv(std * data_var * np.identity(k) + u_sum), mu_sum))

M_prime = torch.mm(u,v.T)
error = 0
for i in range(len(u)):
    for j in user_to_movie_dict[i]:
        error += (float(ratings[(ratings['new_user_index']==i) & (ratings['new_movie_index']==j)].rating) - M_prime[i][j])**2
print('MSE is of Epoch ' + str(epoch_num) + ': ' + str(float(error/len(ratings))))

110.70313000679016
106.21870613098145
